In [1]:
import json
import pandas as pd
import ijson
from datetime import datetime
import time

In [2]:
start_time = time.time()

dict_epochtime = {}
dict_source_ip = {}
dict_dst_ip = {}
dict_source_port = {}
dict_dst_port = {}
dict_stream = {}
dict_sequence = {}
dict_eth_header = {}
dict_ip_header = {}
dict_tcp_header = {}
dict_payload = {}
dict_direction = {}
dict_b_labels ={}
dict_d_labels = {}


total_no = 0
packets = 0
with open("Friday.json", "rb") as f:
    for record in ijson.items(f, "item"):
        total_no += 1
        if 'eth:ethertype:ip:tcp' in record["_source"]["layers"]["frame"]["frame.protocols"]:
            packets += 1
            
            if (packets % 1000000) == 0:
                print((packets/12943381)*100,(time.time() - start_time))
                
            epoch_time = record["_source"]['layers']['frame']['frame.time_epoch']
            source_ip = record["_source"]["layers"]['ip']['ip.src']
            dst_ip = record["_source"]["layers"]['ip']['ip.dst']
            eth_raw = record["_source"]['layers']['eth_raw'][0]
            ip_raw = record["_source"]['layers']['ip_raw'][0]
            tcp_raw = record["_source"]['layers']['tcp_raw'][0]
            all_hex_raw = record["_source"]["layers"]["frame_raw"][0]
            
            all_header_raw = eth_raw + ip_raw + tcp_raw
            
            len_header = len(all_header_raw)
            
            payload_raw = all_hex_raw[len_header:]

            dict_epochtime[packets] = epoch_time
            dict_source_ip[packets] = source_ip
            dict_dst_ip[packets] = dst_ip
            dict_source_port[packets] = record["_source"]['layers']['tcp']['tcp.srcport']
            dict_dst_port[packets] = record["_source"]['layers']['tcp']['tcp.dstport']
            dict_stream[packets] = record["_source"]['layers']['tcp']['tcp.stream']
            dict_sequence[packets] = record["_source"]['layers']['tcp']['tcp.seq']
            dict_eth_header[packets] = eth_raw
            dict_ip_header[packets] = ip_raw
            dict_tcp_header[packets] = tcp_raw
            dict_payload[packets] = payload_raw

            
            if ((source_ip =='192.168.10.12') | (source_ip =='192.168.10.14')|(source_ip =='192.168.10.15')|
                 (source_ip =='192.168.10.17')|(source_ip ==  '192.168.10.5')| (source_ip =='192.168.10.8')| 
                  (source_ip =='192.168.10.9')) & ((dst_ip =='205.174.165.73')|(dst_ip =='52.6.13.28')|
                    (dst_ip =='52.7.235.158')) & (float(epoch_time) >=1499432520) & (float(epoch_time) <= 1499436180):
                    
                    #bot forward=12
                    dict_direction[packets]= 'fwd'
                    dict_d_labels[packets] = 12
                    dict_b_labels[packets] = 1

            elif ((dst_ip =='192.168.10.12') | (dst_ip =='192.168.10.14')|(dst_ip =='192.168.10.15')|
                 (dst_ip =='192.168.10.17')|(dst_ip ==  '192.168.10.5')| (dst_ip =='192.168.10.8')| 
                  (dst_ip =='192.168.10.9')) & ((source_ip =='205.174.165.73')|(source_ip =='52.6.13.28')|
                    (source_ip =='52.7.235.158')) & (float(epoch_time) >=1499432520) & (float(epoch_time) <= 1499436180):
                    
                    #bot backward=12
                    dict_direction[packets]= 'bwd'
                    dict_d_labels[packets] = 12
                    dict_b_labels[packets] = 1                    
                    

            elif  float(epoch_time) >= 1499446500 and float(epoch_time) <= 1499452140:
                ##port scan forward= 13
                if source_ip == '172.16.0.1' and dst_ip == '192.168.10.50':
                    dict_direction[packets]= 'fwd'
                    dict_d_labels[packets] = 13
                    dict_b_labels[packets] = 1 
                ##port scan backward= 13
                elif source_ip == '192.168.10.50' and dst_ip == '172.16.0.1':
                    dict_direction[packets]= 'bwd'
                    dict_d_labels[packets] = 13
                    dict_b_labels[packets] = 1 

            elif float(epoch_time) >= 1499453760 and float(epoch_time) <= 1499454960:
                ##ddos =14
                if source_ip == '172.16.0.1' and dst_ip == '192.168.10.50':
                    dict_direction[packets]= 'fwd'
                    dict_d_labels[packets] = 14
                    dict_b_labels[packets] = 1 
                elif source_ip == '192.168.10.50' and dst_ip == '172.16.0.1':

                    dict_direction[packets]= 'bwd'
                    dict_d_labels[packets] = 14
                    dict_b_labels[packets] = 1 

            
            else:
                dict_direction[packets]= 'unknown'
                dict_d_labels[packets] = 0
                dict_b_labels[packets] = 0

                
print(time.time() - start_time)

7.7259566105641175 257.4408550262451
15.451913221128235 494.7444894313812
23.17786983169235 766.6948356628418
30.90382644225647 1057.8163964748383
38.629783052820585 1367.6279389858246
46.3557396633847 1685.2661125659943
54.08169627394882 1971.105350971222
61.80765288451294 2266.1512382030487
69.53360949507706 2583.5044555664062
2643.7741479873657


In [3]:
pd.DataFrame(dict_d_labels.values()).value_counts()

0     6453606
14    1265657
13     323307
12       9871
dtype: int64

In [4]:
packets

9191727

In [5]:
total_no

9192354

In [6]:
payload_byte_data = pd.DataFrame()

In [7]:
epoch_times_ = pd.DataFrame(dict_epochtime.values())

In [8]:
source_ips_ = pd.DataFrame(dict_source_ip.values())

In [9]:
dst_ips_ = pd.DataFrame(dict_dst_ip.values())

In [10]:
source_ports_ = pd.DataFrame(dict_source_port.values())

In [11]:
dst_ports_ = pd.DataFrame(dict_dst_port.values())

In [12]:
streams_ = pd.DataFrame(dict_stream.values())

In [13]:
seq_ = pd.DataFrame(dict_sequence.values())

In [14]:
eth_ = pd.DataFrame(dict_eth_header.values())

In [15]:
ip_ = pd.DataFrame(dict_ip_header.values())

In [16]:
tcp_ = pd.DataFrame(dict_tcp_header.values())

In [17]:
payload_ = pd.DataFrame(dict_payload.values())

In [18]:
direction_ = pd.DataFrame(dict_direction.values())

In [19]:
d_labels_ = pd.DataFrame(dict_d_labels.values())
b_labels_ = pd.DataFrame(dict_b_labels.values())

In [20]:
payload_byte_data['Epoch Time'] = epoch_times_

In [21]:
payload_byte_data['Source IP'] = source_ips_

In [22]:
payload_byte_data['Dst IP'] = dst_ips_

In [23]:
payload_byte_data['Source Port'] = source_ports_

In [24]:
payload_byte_data['Dst Port'] = dst_ports_

In [25]:
payload_byte_data['Stream Index'] = streams_

In [26]:
payload_byte_data['Sequence Number'] = seq_

In [27]:
payload_byte_data['Eth Header'] = eth_

In [28]:
payload_byte_data['IP Header'] = ip_

In [29]:
payload_byte_data['TCP Header'] = tcp_

In [30]:
payload_byte_data['Payload'] = payload_

In [31]:
payload_byte_data['Direction'] = direction_

In [32]:
payload_byte_data['Binary Label'] = b_labels_

In [33]:
payload_byte_data['Class Label'] = d_labels_

In [34]:
payload_byte_data = payload_byte_data.dropna()

In [35]:
payload_byte_data = payload_byte_data.reset_index(drop = True)

In [36]:
payload_byte_data = payload_byte_data.astype({"Source Port": int, "Dst Port": int})

In [37]:
payload_byte_data.tail()

,Epoch Time,Source IP,Dst IP,Source Port,Dst Port,Stream Index,Sequence Number,Eth Header,IP Header,TCP Header,Payload,Direction,Binary Label,Class Label
8052436,1499454300.224545000,192.168.10.50,172.16.0.1,80,65483,280737,4381,00c1b114eb310019b90a69f10800,45001c57e4c240004006c2f2c0a80a32ac100001,0050ffcb67a3fcb949e931bc501800e593350000,2020202020203c6120687265663d222373636f7065223e...,unknown,0.0,0.0
8052437,1499454300.224548000,192.168.10.50,172.16.0.1,80,65483,280737,11596,00c1b114eb310019b90a69f10800,45000028e4c740004006df1cc0a80a32ac100001,0050ffcb67a418e849e931bc501100e53bb50000,000000000000,unknown,0.0,0.0
8052438,1499454300.224610000,192.168.10.50,172.16.0.1,80,65484,280738,1,00c1b114eb310019b90a69f10800,45002d7356e9400040063fb0c0a80a32ac100001,0050ffcc76d259cb355a61d7501800e5a4510000,485454502f312e3120323030204f4b0d0a446174653a20...,unknown,0.0,0.0
8052439,1499454300.224615000,192.168.10.50,172.16.0.1,80,65484,280738,11596,00c1b114eb310019b90a69f10800,4500002856f1400040066cf3c0a80a32ac100001,0050ffcc76d28716355a61d7501100e5a2cb0000,000000000000,unknown,0.0,0.0
8052440,1499454300.224733000,192.168.10.50,172.16.0.1,80,65485,280739,1,00c1b114eb310019b90a69f10800,45002d7308ba400040068ddfc0a80a32ac100001,0050ffcd6e6e079a800de44c501800e5a4510000,485454502f312e3120323030204f4b0d0a446174653a20...,unknown,0.0,0.0


In [38]:
len(payload_byte_data)

8052441

In [39]:
payload_byte_data['Direction'].value_counts()

unknown    6453606
fwd         922153
bwd         676682
Name: Direction, dtype: int64

In [40]:
payload_byte_data.groupby('Direction')['Class Label'].value_counts()

Direction  Class Label
bwd        14.0            510922
           13.0            160677
           12.0              5083
fwd        14.0            754735
           13.0            162630
           12.0              4788
unknown    0.0            6453606
Name: Class Label, dtype: int64

In [41]:
df_b = payload_byte_data[payload_byte_data['Class Label']==0]

In [42]:
df_a = payload_byte_data[payload_byte_data['Class Label'] != 0]

In [43]:
df_b = df_b.reset_index(drop = True)

In [44]:
df_a = df_a.reset_index(drop = True)

In [45]:
df1 = pd.read_csv('troubleshooting dataset/Friday-WorkingHours.csv')

In [46]:
def df_prep_frwrd_prts(df1):
    
    df1 = df1[df1['Protocol'] ==6]
    df1 = df1[df1['Label'] =='BENIGN']
    #df1 = df1[['Src IP', 'Dst IP', 'Src Port', 'Dst Port', 'Timestamp','Label']].copy()
    
    #unique src prt and dst prt
    srcprt_list = df1['Src Port'].unique()
    dstprt_list = df1['Dst Port'].unique()
    print(len(srcprt_list))
    print('-----------')
    print(len(dstprt_list))
    print('-----------')
    intersect = set(srcprt_list).intersection(dstprt_list)
    print(len(intersect))
    print('-----------')
    #print(intersect)
    
    
    df1 = df1[~df1['Src Port'].isin(intersect)]
    
    #calculate the epoch time from timestamp
    #df1['epoch']=df1['Timestamp'].apply(lambda x: (datetime.strptime(x, '%d/%m/%Y %H:%M')) if x.count(':') == 1 else  (datetime.strptime(x, '%d/%m/%Y %H:%M:%S')))
    #df1['epoch']=df1['epoch'].apply(lambda x: int((datetime(x.year,x.month,x.day,(x.hour+12),x.minute,x.second)).timestamp()) if (x.hour>=1)&(x.hour<=7) else int((datetime(x.year,x.month,x.day,x.hour,x.minute,x.second)).timestamp()))
    
    df1 = df1.reset_index(drop=True)
    
    return df1

In [47]:
df1 = df_prep_frwrd_prts(df1)

32086
-----------
1570
-----------
911
-----------


In [48]:
df_b_fwd = df_b[df_b['Source Port'].isin(df1['Src Port'].unique())] 

In [49]:
df_b_bwd = df_b[df_b['Dst Port'].isin(df1['Src Port'].unique())] 

In [50]:
df_b_fwd['Direction'] = df_b_fwd['Direction'].replace(['unknown'], 'fwd')

C:\Users\Jalal\AppData\Local\Temp\ipykernel_22404\3752041767.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_b_fwd['Direction'] = df_b_fwd['Direction'].replace(['unknown'], 'fwd')


In [51]:
df_b_bwd['Direction'] = df_b_bwd['Direction'].replace(['unknown'], 'bwd')

C:\Users\Jalal\AppData\Local\Temp\ipykernel_22404\3888835064.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_b_bwd['Direction'] = df_b_bwd['Direction'].replace(['unknown'], 'bwd')


In [52]:
df_b_bwd

,Epoch Time,Source IP,Dst IP,Source Port,Dst Port,Stream Index,Sequence Number,Eth Header,IP Header,TCP Header,Payload,Direction,Binary Label,Class Label
2,1499428790.315459000,192.168.10.3,192.168.10.50,3268,56108,0,1,0019b90a69f11866da9be37d0800,4500007c5ff6400080060500c0a80a03c0a80a32,0cc4db2c3630dfa96a70feef8018081ecac30000010108...,00000044604206092a864886f712010202020111001000...,bwd,0.0,0.0
3,1499428790.315463000,192.168.10.3,192.168.10.50,3268,56108,0,1,0019b90a69f11866da9be37d0800,4500007c5ff6400080060500c0a80a03c0a80a32,0cc4db2c3630dfa96a70feef8018081ecac30000010108...,00000044604206092a864886f712010202020111001000...,bwd,0.0,0.0
8,1499428790.316526000,192.168.10.3,192.168.10.50,389,42144,1,1,0019b90a69f11866da9be37d0800,450001705ff740008006040bc0a80a03c0a80a32,0185a4a0ff37c390541759938018081e0c130000010108...,000001386082013406092a864886f71201020202011100...,bwd,0.0,0.0
9,1499428790.316531000,192.168.10.3,192.168.10.50,389,42144,1,1,0019b90a69f11866da9be37d0800,450001705ff740008006040bc0a80a03c0a80a32,0185a4a0ff37c390541759938018081e0c130000010108...,000001386082013406092a864886f71201020202011100...,bwd,0.0,0.0
14,1499428806.714219000,192.168.10.3,192.168.10.50,3268,56108,0,73,0019b90a69f11866da9be37d0800,4500007c5ff84000800604fec0a80a03c0a80a32,0cc4db2c3630dff16a7100828018081cdcfc0000010108...,00000044604206092a864886f712010202020111001000...,bwd,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6453484,1499454300.212460000,172.217.12.205,192.168.10.14,443,58093,280715,3982,b8ac6f3607ee00c1b114eb310800,450000f7dfe60000340621beacd90ccdc0a80a0e,01bbe2eda84cbf47d787ec2350180169decb0000,17030300ca0000000000000003b9423a0b5901a1d34005...,bwd,0.0,0.0
6453486,1499454300.212688000,172.217.12.205,192.168.10.14,443,58093,280715,4189,b8ac6f3607ee00c1b114eb310800,4500019ddfe7000034062117acd90ccdc0a80a0e,01bbe2eda84cc016d787ec2350180169e0870000,1703030170000000000000000427cfa8c91d47cda4d459...,bwd,0.0,0.0
6453488,1499454300.212736000,172.217.12.205,192.168.10.14,443,58093,280715,4562,b8ac6f3607ee00c1b114eb310800,450000acdfe8000034062207acd90ccdc0a80a0e,01bbe2eda84cc18bd787ec235018016903ef0000,170303007f000000000000000529465b5b09e2c6a7bf19...,bwd,0.0,0.0
6453489,1499454300.212738000,172.217.12.205,192.168.10.14,443,58093,280715,4694,b8ac6f3607ee00c1b114eb310800,45000056dfe900003406225cacd90ccdc0a80a0e,01bbe2eda84cc20fd787ec2350180169e55a0000,170303002900000000000000066521f26029ac678a3244...,bwd,0.0,0.0


In [53]:
df_all = pd.concat([df_a, df_b_fwd , df_b_bwd])

In [54]:
df_all = df_all.reset_index(drop = True)

In [55]:
df_all

,Epoch Time,Source IP,Dst IP,Source Port,Dst Port,Stream Index,Sequence Number,Eth Header,IP Header,TCP Header,Payload,Direction,Binary Label,Class Label
0,1499432653.990571000,192.168.10.9,205.174.165.73,1841,8080,13099,0,00c1b114eb31b8ac6f1d1f6c0800,450000346c8f40008006508bc0a80a09cdaea549,07311f905c691851000000008002200075eb0000020405...,,fwd,1.0,12.0
1,1499432653.991397000,205.174.165.73,192.168.10.9,8080,1841,13099,0,b8ac6f1d1f6c00c1b114eb310800,45000034000040003e06ff1acdaea549c0a80a09,1f90073113dd44cb5c69185280127210cb220000020405...,,bwd,1.0,12.0
2,1499432653.991520000,192.168.10.9,205.174.165.73,1841,8080,13099,1,00c1b114eb31b8ac6f1d1f6c0800,450000286c90400080065096c0a80a09cdaea549,07311f905c69185213dd44cc501001007d050000,000000000000,fwd,1.0,12.0
3,1499432653.992542000,192.168.10.9,205.174.165.73,1841,8080,13099,1,00c1b114eb31b8ac6f1d1f6c0800,450000ea6c91400080064fd3c0a80a09cdaea549,07311f905c69185213dd44cc50180100307c0000,474554202f6170692f706f703f626f7469643d6d697461...,fwd,1.0,12.0
4,1499432653.993057000,205.174.165.73,192.168.10.9,8080,1841,13099,1,b8ac6f1d1f6c00c1b114eb310800,45000028822a40003e067cfccdaea549c0a80a09,1f90073113dd44cc5c691914501000ed7c560000,000000000000,bwd,1.0,12.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5260217,1499454300.212460000,172.217.12.205,192.168.10.14,443,58093,280715,3982,b8ac6f3607ee00c1b114eb310800,450000f7dfe60000340621beacd90ccdc0a80a0e,01bbe2eda84cbf47d787ec2350180169decb0000,17030300ca0000000000000003b9423a0b5901a1d34005...,bwd,0.0,0.0
5260218,1499454300.212688000,172.217.12.205,192.168.10.14,443,58093,280715,4189,b8ac6f3607ee00c1b114eb310800,4500019ddfe7000034062117acd90ccdc0a80a0e,01bbe2eda84cc016d787ec2350180169e0870000,1703030170000000000000000427cfa8c91d47cda4d459...,bwd,0.0,0.0
5260219,1499454300.212736000,172.217.12.205,192.168.10.14,443,58093,280715,4562,b8ac6f3607ee00c1b114eb310800,450000acdfe8000034062207acd90ccdc0a80a0e,01bbe2eda84cc18bd787ec235018016903ef0000,170303007f000000000000000529465b5b09e2c6a7bf19...,bwd,0.0,0.0
5260220,1499454300.212738000,172.217.12.205,192.168.10.14,443,58093,280715,4694,b8ac6f3607ee00c1b114eb310800,45000056dfe900003406225cacd90ccdc0a80a0e,01bbe2eda84cc20fd787ec2350180169e55a0000,170303002900000000000000066521f26029ac678a3244...,bwd,0.0,0.0


In [56]:
df_all.groupby('Direction')['Class Label'].value_counts()

Direction  Class Label
bwd        0.0            1950805
           14.0            510922
           13.0            160677
           12.0              5083
fwd        0.0            1710582
           14.0            754735
           13.0            162630
           12.0              4788
Name: Class Label, dtype: int64

In [57]:
df_all.to_pickle("Friday_Packet_Level_Data.pkl")